#Main file
This main file is based on the scripts that I had for minimizing the machines from Netlogo. The original was made to capture output from Behaviour Space, processs the machines and then print it to use in Stata. This is too cumbersome, so decided to implement and do everything in Python so I can centralise all the analysis and work on the next algorithms such as Joint Machines, frequencies and unused behavioural states in order to analyse properly how the transitions are happening.

#1 Receiving the Netlogo output
The first step is to load the files. Also, as the global variables of interest in order to associate it with the output name file.

In [1]:
import pandas as pd
import numpy as np
import copy
import minimization as minz #My script in same folder for minimization routines

#Choose here the Globals and Name used for the experiment to load.
#Make sure the files exist (i.e. from Netlogo simulations)
experiment_clue = "trial1"
n_states = 4
n_signals = 0
n_rounds = 20
N = 40
n_parents = 20

globals_list = (experiment_clue, n_states, n_signals, n_rounds, N, n_parents) #Save them as a list

#Experiment name based on the chosen experiment_clue and globals
chosen_experiment = "%s_states_%s_signal_%s_rounds_%s_N_%s_parents_%s.txt" % globals_list

#Both files have to use the same "chosen experiment" (to make sure come from the same simulation in Netlogo)
summary_file_name = "summary_" + chosen_experiment #Summary output
strategies_file_name = "strategies_" + chosen_experiment #Strategies output

#Path to Netlogo outputs
netlogo_folder = "/Users/luisalejandrolee/Dropbox/Thesis Phd/\
Coordination autos Chapter three/outputs_to_text/" #Netlogo outputs in this folder

#Read files and save them as data
df_sum = pd.read_csv(netlogo_folder + summary_file_name)
df_strat = pd.read_csv(netlogo_folder + strategies_file_name)

#2 Minimise the automata
Use the functions to have a simple code here for minimising the auto and storing other relevant variables (as available states, etc)

In [2]:
#Other required globals
n_obs = 2 if n_signals == 0 else 4 #Define here (or change) the possible observations of the machines

canon_autos_list = [] #Create empty lists to store the processed autos below
min_autos_list = []


for i in df_strat.index: #For each row...
    netlogo_auto = df_strat.auto_long[i] #... for all netlogo_auto
    big_auto = minz.to_format_netlogo_auto(netlogo_auto) #Use function to convert the raw Netlogo auto in a list format
    init_state = big_auto[0] #Save initial state of the machine
    normal_auto = minz.new_empty_auto(n_obs, n_states) #Use function to create a new empty auto as a numpy array

    # Next block it to fill the new 'normal_auto' with the information from big_auto.
    # The objective is that normal_auto=big_auto but as an array (instead of a list)
    my_index = xrange(1, len(big_auto), n_obs + 1) # Each number in the index is where a state starts
    for i, j in enumerate(my_index):
        normal_auto['actions'][i] = big_auto[j]
        normal_auto['transitions'][i] = big_auto[j + 1:j + n_obs + 1]
        
    canon_auto = minz.convert_to_canonical(normal_auto, n_states, init_state, n_obs) #Use function for canonical form
    
    access_states = len(canon_auto) #n_states now is only the accesible states of the machine (before minimization)
    
    #Use function to get minimum behavioural equivalent auto
    #Passes "0" as 3rd argument because that's init_state now (always 0 for canonical auto)
    min_auto = minz.minimized_automaton(canon_auto, access_states, 0, n_obs)
    
    #Update autos lists
    canon_autos_list.append(canon_auto) #Store proccessed autos in the corresponding list
    min_autos_list.append(min_auto)
    
#Add the processed autos lists as columns to df_strat
df_strat["canon_autos"] = canon_autos_list #Add the lists with autos to the dataframe
df_strat["min_autos"] = min_autos_list

#3)  Variables key for analysis

##3.1 Accesible states and minimum behavioural states

In [3]:
#Accesible states

df_strat["access_states"] = -1 #Initialise negative to capture errors later
for i in df_strat.index: #For all canonical autos
    states = len(df_strat.canon_autos[i]) #Accesible states is the lenght of the canonical auto
    df_strat.loc[i, "access_states"] = states #Save to dataframe

#Minimum behavioral states

df_strat["min_states"] = -1 #Initialise negative to capture errors later
for i in df_strat.index: #For all canonical autos
    states = len(df_strat.min_autos[i]) #Min states is the lenght of the min auto, taken as proxy for complexity
    df_strat.loc[i, "min_states"] = states #Save to dataframe

##3.2 Joint machine (not minimized) and Unused States measure

In [4]:
#Prepare dataframe to keep track of unused states
df_strat["used_states"] = 0 # Will contain a list with states of each min_auto
used_states_list = []
for i, auto in enumerate(df_strat.min_autos): #all minimized autos
    a = [0 for ix in xrange(df_strat.min_states[i])] #List the size of minimised machine's states
    used_states_list.append(a)
df_strat["used_states"] = used_states_list #Add to dataframe

# Lists to keep track of joint machines
gen_list = []
jm_list = []

#Create the joint machines (not minimized) and track unused states
#Main code of this part

for gen in df_sum.generation: #each generation
    print "generation = ", gen #Useful for debugging (keeping track of simulation time)
    #Next line (the "for" block) is tricky:
    #It creates a dataframe containing only the autos with population = column and current generation.
    #Uses "iterrows()" to iterate over the index of the dataframe (df_strat), keeping it on i0, which is
    #needed to acces later the particular auto that was used for the joint machine (accesed by row0.min_autos)
    #This is used (instead of a simple enumerate) to access that row later when updating the used states 
    #Same logic for the second "for" block, but for population row
    
    for i0, row0 in df_strat[(df_strat.population == "column") & (df_strat.generation == gen)].iterrows():
        for i1, row1 in df_strat[(df_strat.population == "row") & (df_strat.generation == gen)].iterrows():
            #print "generation = ", gen, "i0 = ", i0, "i1 = ", i1
            
            #Initialize variables
            size = len(row0.min_autos) * len (row1.min_autos) + 1  #Max states before joint machine cycles
            states = [([None] * size), ([None] * size)]
            actions = [([None] * size), ([None] * size)]
            auto0_state = 0 #Current state of the machines (intial is zero)
            auto1_state = 0

            metastate = -1
            cyclestart = -1
            cycle = False

            #Detect cycle (i.e. when the same pair of states appears) 
            while not cycle:
                metastate += 1
                states[0][metastate] = auto0_state #Add current state of autos
                states[1][metastate] = auto1_state
                actions[0][metastate] = row0.min_autos[auto0_state][0] #Add current action of autos (based on current state)
                actions[1][metastate] = row1.min_autos[auto1_state][0] #Remember row1.min_autos is the minimized auto

                #Move autos to next state based on action of other
                auto0_state = minz.update_state_no_signal(row0.min_autos, auto0_state, actions[1][metastate])
                auto1_state = minz.update_state_no_signal(row1.min_autos, auto1_state, actions[0][metastate])

                #Now cycle to check if we have been at the two new states before
                for ms in xrange(metastate): #previous states
                    if (states[0][metastate] == states [0][ms]) and (states[1][metastate] == states [1][ms]):#cycle has started
                        cyclestart = ms
                        cycle = True
                        
            #Save joint machine and generation as list (to later make them a dataframe)
            jm = {"metastate": metastate, "cyclestart": cyclestart, "states": states, "actions": actions} #joint machine
            gen_list.append(gen) #Keeps track of generation
            jm_list.append(jm)   #Keeps track of joint machines
            """
            print "auto0 = ", row0.min_autos
            print "auto1 = ", row1.min_autos
            print jm
            """
            #Update used_states:
            for st in jm["states"][0]: #for states visited by auto0
                if st != None:
                    df_strat["used_states"][i0][st] = 1 # 1 for states visited. Unvisited remain 0
            
            for st in jm["states"][1]: #for states visited by auto1
                if st != None:
                    df_strat["used_states"][i1][st] = 1


#Store the joint machines and generation (a new dataframe)
df_jms = pd.DataFrame(columns = ("generation", "jm")) #Store joint machines with associated generation
df_jms.generation = gen_list
df_jms.jm = jm_list


#Seems to be working fine (just don't print!). Check with a couple of autos
#Add the unused states to each machine (in df_strat)
#Send some things to minimization script
#Start minimization of joint machines (once everything is checked properly)



generation =  0
generation =  1
generation =  2
generation =  3
generation =  4
generation =  5
generation =  6
generation =  7
generation =  8
generation =  9
generation =  10
generation =  11
generation =  12
generation =  13
generation =  14
generation =  15
generation =  16
generation =  17
generation =  18
generation =  19
generation =  20
generation =  21
generation =  22
generation =  23
generation =  24
generation =  25
generation =  26
generation =  27


In [5]:
df_strat

,generation,ID,population,score,auto_long,ce_individual,canon_autos,min_autos,access_states,min_states,used_states
0,0,4,column,1.24000,[0 A 0 1 A 2 0 B 1 2 A 3 2],0,"[[A, [0, 1]], [A, [2, 0]], [B, [1, 2]]]","[[A, [0, 1]], [A, [2, 0]], [B, [1, 2]]]",3,3,"[1, 1, 1]"
1,0,59,column,1.08250,[0 A 1 0 A 2 1 B 0 0 B 3 3],0,"[[A, [1, 0]], [A, [2, 1]], [B, [0, 0]]]","[[A, [1, 0]], [A, [2, 1]], [B, [0, 0]]]",3,3,"[1, 1, 1]"
2,0,78,column,1.42750,[2 A 2 0 A 1 1 B 0 1 A 2 0],0,"[[B, [1, 2]], [A, [0, 1]], [A, [2, 2]]]","[[B, [1, 2]], [A, [0, 1]], [A, [2, 2]]]",3,3,"[1, 1, 1]"
3,0,62,row,1.35875,[2 B 1 0 B 3 1 B 2 0 A 2 1],0,"[[B, [0, 1]], [B, [2, 1]], [B, [3, 2]], [A, [0...","[[B, [0, 1]], [B, [2, 1]], [B, [3, 2]], [A, [0...",4,4,"[1, 1, 1, 1]"
4,0,26,row,0.84750,[3 B 2 2 B 0 1 B 1 3 A 1 3],0,"[[A, [1, 0]], [B, [2, 1]], [B, [3, 3]], [B, [1...","[[A, [1, 0]], [B, [2, 1]], [B, [3, 3]], [B, [1...",4,4,"[1, 1, 1, 1]"
5,0,70,column,1.46250,[3 A 3 1 A 3 3 B 0 0 B 0 3],0,"[[B, [1, 0]], [A, [0, 2]], [A, [0, 0]]]","[[B, [1, 0]], [A, [0, 2]], [A, [0, 0]]]",3,3,"[1, 1, 1]"
6,0,76,row,1.13500,[1 A 2 0 B 3 2 A 0 2 B 0 3],0,"[[B, [1, 3]], [B, [2, 1]], [A, [3, 2]], [A, [2...","[[B, [1, 2]], [B, [2, 1]], [A, [2, 2]]]",4,3,"[1, 1, 1]"
7,0,3,column,1.10875,[2 B 1 1 A 0 1 A 2 1 B 1 1],0,"[[A, [0, 1]], [A, [2, 1]], [B, [1, 1]]]","[[A, [0, 1]], [A, [2, 1]], [B, [1, 1]]]",3,3,"[1, 1, 1]"
8,0,23,column,0.73125,[2 A 2 0 B 0 0 B 2 3 A 3 0],0,"[[B, [0, 1]], [A, [1, 2]], [A, [0, 2]]]","[[B, [0, 1]], [A, [1, 2]], [A, [0, 2]]]",3,3,"[1, 1, 1]"
9,0,31,column,0.95750,[2 B 1 1 B 0 1 B 3 1 A 0 2],0,"[[B, [1, 3]], [A, [2, 0]], [B, [3, 3]], [B, [2...","[[B, [1, 2]], [A, [2, 0]], [B, [2, 2]]]",4,3,"[1, 1, 1]"


In [ ]:

#FOR EXPLAINING STUFF


# unused behavior measures (using used states list)
#joint machine frequencies
#other machines frequencies (minimised)
#regime identification (using joint machines)
#regime change identification
#Graphs and stats (automate them)

##Next:
#1) describe graphically his transition explanations


